# Fine-tune FLAN-T5 XL/XXL using DeepSpeed & Hugging Face Transformers

FLAN-T5, released with the [Scaling Instruction-Finetuned Language Models](https://arxiv.org/pdf/2210.11416.pdf) paper, is an enhanced version of T5 that has been fine-tuned in a mixture of tasks, or simple words, a better T5 model in any aspect. FLAN-T5 outperforms T5 by double-digit improvements for the same number of parameters. Google has open sourced [5 checkpoints available on Hugging Face](https://huggingface.co/models?other=arxiv:2210.11416) ranging from 80M parameter up to 11B parameter.

In a previous blog post, we already learned how to [“Fine-tune FLAN-T5 for chat & dialogue summarization”](https://www.philschmid.de/fine-tune-flan-t5) using [the base version (250M parameter)](https://huggingface.co/google/flan-t5-base) of the model. In this blog post, we look into how we can scale the training from the Base version to the [XL (3B)](https://huggingface.co/google/flan-t5-xl) or [XXL (11B)](https://huggingface.co/google/flan-t5-xxl). 

This means we will learn how to fine-tune FLAN-T5 XL & XXL using model parallelism, multiple GPUs, and [DeepSpeed ZeRO](https://www.deepspeed.ai/tutorials/zero/). 

in addition to the tutorial, we have run a series of experiments to help you choose the right hardware setup. You can find the details in the Results & Experiments section.

In [1]:
# install git lfs for pushing artifacts
!sudo apt install git-lfs
# install torch with the correct cuda version, check nvcc --version
!pip install torch --extra-index-url https://download.pytorch.org/whl/cu116 --upgrade
# install Hugging Face Libraries
!pip install "transformers==4.26.0" "datasets==2.9.0" "accelerate==0.16.0" "evaluate==0.4.0" --upgrade
# install deepspeed and ninja for jit compilations of kernels
!pip install "deepspeed==0.8.0" ninja --upgrade
# install additional dependencies needed for training
!pip install rouge-score nltk py7zr tensorboard

# Process dataset

Similar to the [“Fine-tune FLAN-T5 for chat & dialogue summarization”](https://www.philschmid.de/fine-tune-flan-t5) we need to prepare a dataset to fine-tune our model. As mentioned in the beginning, we will fine-tune [FLAN-T5-XXL](https://huggingface.co/google/flan-t5-xxl) on the [CNN Dailymail Dataset](https://huggingface.co/datasets/cnn_dailymail). The blog post is not going into detail about the dataset generation. If you want to learn the detailed steps check out the [previous post](https://www.philschmid.de/fine-tune-flan-t5). 

We define some parameters, which we use throughout the whole example, feel free to adjust it to your needs.

In [1]:
# experiment config
model_id = "google/flan-t5-xxl" # Hugging Face Model Id
dataset_id = "cnn_dailymail" # Hugging Face Dataset Id
dataset_config = "3.0.0" # config/verison of the dataset
save_dataset_path = "data" # local path to save processed dataset
text_column = "article" # column of input text is
summary_column = "highlights" # column of the output text 
# custom instruct prompt start
prompt_template = f"Summarize the following news article:\n{{input}}\nSummary:\n"

Compared to the [previous example](https://www.philschmid.de/fine-tune-flan-t5), we are splitting the processing and training into two separate paths. This allows you to run the preprocessing outside of the GPU instance. We process (tokenize) the dataset and save it to disk and then load in our train script from disk again.

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
import numpy as np 

# Load dataset from the hub
dataset = load_dataset(dataset_id,name=dataset_config)
# Load tokenizer of FLAN-t5-base
tokenizer = AutoTokenizer.from_pretrained(model_id)

print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['test'])}")

# Train dataset size: 287113
# Test dataset size: 11490

We defined a `prompt_template` in our config, which we will use to construct an instruct prompt for better performance of our model. Our `prompt_template` has a “fixed” start and end, and our document is in the middle. This means we need to ensure that the “fixed” template parts + document are not exceeding the max length of the model. Therefore we calculate the max length of our document, which we will later use for padding and truncation

In [3]:
prompt_length = len(tokenizer(prompt_template.format(input=""))["input_ids"])
max_sample_length = tokenizer.model_max_length - prompt_length
print(f"Prompt length: {prompt_length}")
print(f"Max input length: {max_sample_length}")

# Prompt length: 12
# Max input length: 500

Prompt length: 12
Max input length: 500


We know now that our documents can be “500” tokens long to fit our `template_prompt` still correctly. In addition to our input, we need to understand better our “target” sequence length meaning and how long are the summarization ins our dataset. Therefore we iterate over the dataset and calculate the max input length (at max 500) and the max target length. (takes a few minutes)

In [4]:
from datasets import concatenate_datasets
import numpy as np


# The maximum total input sequence length after tokenization. 
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x[text_column], truncation=True), batched=True, remove_columns=[text_column, summary_column])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
max_source_length = min(max_source_length, max_sample_length)
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization. 
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x[summary_column], truncation=True), batched=True, remove_columns=[text_column, summary_column])
target_lenghts = [len(x) for x in tokenized_targets["input_ids"]]
# use 95th percentile as max target length
max_target_length = int(np.percentile(target_lenghts, 95))
print(f"Max target length: {max_target_length}")

  0%|          | 0/299 [00:00<?, ?ba/s]

Max source length: 500


  0%|          | 0/299 [00:00<?, ?ba/s]

Max target length: 129


We now have everything needed to process our dataset.

In [ ]:
import os

def preprocess_function(sample, padding="max_length"):
    # created prompted input
    inputs = [prompt_template.format(input=item) for item in sample[text_column]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=tokenizer.model_max_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample[summary_column], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# process dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=list(dataset["train"].features))

# save dataset to disk
tokenized_dataset["train"].save_to_disk(os.path.join(save_dataset_path,"train"))
tokenized_dataset["test"].save_to_disk(os.path.join(save_dataset_path,"eval"))

## Fine-tune model using `deepspeed`

Done! We can now start training our model! We learned in the introduction that we would leverage the DeepSpeed integration with the Hugging Face Trainer. Therefore we need to create a `deespeed_config.json`. In the [DeepSpeed Configuration,](https://www.deepspeed.ai/docs/config-json/) we define the ZeRO strategy we want to use and if we want to use mixed precision training. The Hugging Face Trainer allows us to inherit values from the `TrainingArguments` in our `deepspeed_config.json` to avoid duplicate values, check the [documentation for more information.](https://huggingface.co/docs/transformers/v4.26.1/en/main_classes/deepspeed#configuration)

We created 4 deepspeed configurations for the experiments we ran, including `CPU offloading` and `mixed precision`: 

- [ds_flan_t5_z3_config.json](./configs/ds_flan_t5_z3_config.json)
- [ds_flan_t5_z3_config_bf16.json](./configs/ds_flan_t5_z3_config_bf16.json)
- [ds_flan_t5_z3_offload.json](./configs/ds_flan_t5_z3_offload.json)
- [ds_flan_t5_z3_offload_bf16.json](./configs/ds_flan_t5_z3_offload_bf16.json)

Depending on your setup, you can use those, e.g. if you are running on NVIDIA V100s, you have to use the config without `bf16` since V100 are not support `bfloat16` types. 

> When fine-tuning `T5` models we cannot use `fp16` since it leads to overflow issues, see: [#4586](https://github.com/huggingface/transformers/issues/4586), [#10830](https://github.com/huggingface/transformers/issues/10830), [#10956](https://github.com/huggingface/transformers/pull/10956)
> 

As mentioned in the beginning, we are using a p4dn.24xlarge AWS EC2 Instance including 8x NVIDIA A100 40GB. This means we can leverage `bf16`, which reduces the memory footprint of the model by almost ~2x, which allows us to train without offloading efficiently. 

We are going to use the [ds_flan_t5_z3_config_bf16.json](./configs/ds_flan_t5_z3_config_bf16.json). If you are irritated by the `auto` values, check the [documentation](https://huggingface.co/docs/transformers/v4.26.1/en/main_classes/deepspeed#configuration).

In [16]:
!deepspeed --num_gpus=8 scripts/run_seq2seq_deepspeed.py \
    --model_id $model_id \
    --dataset_path $save_dataset_path \
    --epochs 3 \
    --per_device_train_batch_size 8 \
    --per_device_eval_batch_size 8 \
    --generation_max_length $max_target_length \
    --lr 1e-4 \
    --deepspeed configs/ds_flan_t5_z3_config_bf16.json 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
deepspeed --num_gpus=8 scripts/run_seq2seq_deepspeed.py --model_id google/flan-t5-xxl --dataset_path data --epochs 3 --per_device_train_batch_size 8 --per_device_eval_batch_size 8 --generation_max_length 129 --lr 1e-4 --deepspeed configs/ds_flan_t5_z3_config_bf16.json


# Results

dataset: `"cnn_dailymail"`
-  training examples: `287113`
-  validation examples: `13368`

hyperparamters:
- epochs: `3`
- learning rate: `1e-4`

setups: 
- 4x V100 16GB: p3.8xlarge
- 4x A10G 24GB: g5.24xlarge
- 8x V100 16GB: p3.16xlarge
- 8x A100 40GB: p4dn.24xlarge


| Model             | DS offload | Hardware     | batch size per GPU | precision | duration | cost   |
|-------------------|------------|--------------|--------------------|-----------|----------|--------|
| FLAN-T5-XL (3B)   | No         | 4x V100 16GB | OOM                | fp32      | -        | -      |
| FLAN-T5-XL (3B)   | No         | 8x V100 16GB | 1                  | fp32      | 105h     | ~$2570 |
| FLAN-T5-XL (3B)   | No         | 8x A100 40GB | 72                 | bf16     |   2,5h       | ~$81       |
| FLAN-T5-XL (3B)   | Yes        | 4x V100 16GB | 8                  | fp32      | 69h      | ~$828  |
| FLAN-T5-XL (3B)   | Yes        | 8x V100 16GB | 8                  | fp32      | 32h      | ~$768  |
| FLAN-T5-XXL (11B) | No        | 8x A100 40GB | 8                | bf16      | 10h        | ~$322      |
| FLAN-T5-XXL (11B) | Yes        | 4x V100 16GB | OOM                | fp32      | -        | -      |
| FLAN-T5-XXL (11B) | Yes        | 8x V100 16GB | OOM                | fp32      | -        | -      |
| FLAN-T5-XXL (11B) | Yes        | 4x A10G 24GB | 24                | bf16      | 90h      | ~$732  |
| FLAN-T5-XXL (11B) | Yes        | 8x A100 40GB | 48                | bf16      | 19h      | ~$613  |